In [2]:
import json
import requests
from openai import OpenAI

In [ ]:
response = requests.post(
    'http://localhost:11434/api/generate',
    json = {
        'model': 'qwen3:8b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

response.raise_for_status()

Run time for the 8b model was 3 min 40 sec

In [ ]:
data = response.json()

In [ ]:
data["response"]

In [ ]:
data

In [ ]:
response = requests.post(
    'http://localhost:11434/api/generate',
    json = {
        'model': 'qwen3:14b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

response.raise_for_status()

trying to run the 14b model took 9 min 16 sec, perhaps I need to check the GPU driver on my laptop?

Curious to see what the difference may be if I offload this to Coruscant?

In [ ]:
data = response.json()

In [ ]:
data['response']

In [ ]:
response = requests.post(
    'http://coruscant.local:11434/api/generate',
    json = {
        'model': 'qwen3:14b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

Well ok then, running on Coruscant returned 3x faster for the 14b model...
Second run is about just as fast 3 min 48 seconds

In [ ]:
data = response.json()

In [ ]:
print(data['response'])

The issue I was having with previous output was that the string needed to be printed by python first. By extracting it "raw" it was just a representation of the string with added characters. 

Curious to see how fast the 8b model performs on Coruscant

In [ ]:
response = requests.post(
    'http://coruscant.local:11434/api/generate',
    json = {
        'model': 'qwen3:8b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

First run... 32 seconds!

In [ ]:
data = response.json()

In [ ]:
print(data['response'])

In [ ]:
def get_response(prompt: str, model: str = 'qwen3:8b', ctx: int = 10000) -> str:
    response = requests.post(
    'http://coruscant.local:11434/api/generate',
        json = {
            'model': model,
            'prompt': prompt,
            'stream': False,
            'options': {
                'num_ctx': ctx
            }
        }
    )

    data = response.json()

    if 'response' not in data:
        print('Unexpected output:', data)
        return ""
    
    return data['response']

From what I have seen it is possible to utilize the OpenAI method/syntax while still targeting ollama

In [2]:
base_url = 'http://coruscant.local:11434/v1'

ollama = OpenAI(base_url=base_url, api_key='ollama')

In [3]:
response = ollama.chat.completions.create(model= 'qwen3:8b', messages= [{'role': 'user', 'content': 'What are decorators in Python?'}])

response.choices[0].message.content

'Decorators in Python are a powerful feature that allows you to modify or enhance functions, methods, or classes without altering their original code. They act as **wrappers** that add functionality to existing code, promoting modularity and separation of concerns. Here\'s a structured explanation:\n\n---\n\n### **Key Concepts**\n1. **Decorator Syntax**:\n   - The `@` symbol is used to apply a decorator to a function or class.\n   - Example:\n     ```python\n     @decorator_function\n     def my_function():\n         pass\n     ```\n   - This is equivalent to:\n     ```python\n     def my_function():\n         pass\n     my_function = decorator_function(my_function)\n     ```\n\n2. **Decorator Structure**:\n   - A decorator is typically a **function or class** that takes the target function as an argument and returns a **new function** that wraps the original.\n   - Example:\n     ```python\n     def my_decorator(func):\n         def wrapper():\n             print("Before function call

In [4]:
print(response.choices[0].message.content)

Decorators in Python are a powerful feature that allows you to modify or enhance functions, methods, or classes without altering their original code. They act as **wrappers** that add functionality to existing code, promoting modularity and separation of concerns. Here's a structured explanation:

---

### **Key Concepts**
1. **Decorator Syntax**:
   - The `@` symbol is used to apply a decorator to a function or class.
   - Example:
     ```python
     @decorator_function
     def my_function():
         pass
     ```
   - This is equivalent to:
     ```python
     def my_function():
         pass
     my_function = decorator_function(my_function)
     ```

2. **Decorator Structure**:
   - A decorator is typically a **function or class** that takes the target function as an argument and returns a **new function** that wraps the original.
   - Example:
     ```python
     def my_decorator(func):
         def wrapper():
             print("Before function call")
             func()
     

Working on the Wikipedia Function

In [37]:
import wikipediaapi

In [38]:
wiki_obj = wikipediaapi.Wikipedia(user_agent='TopicSearch (grant.mlanier@gmail.com)', language='en')

In [39]:
wave = wiki_obj.page('The Great Wave off Kanagawa')

In [30]:
wave.exists()

True

In [31]:
print(wave.summary)

The Great Wave off Kanagawa (Japanese: 神奈川沖浪裏, Hepburn: Kanagawa-oki Nami Ura; lit. 'Under the Wave off Kanagawa') is a woodblock print by the Japanese ukiyo-e artist Hokusai (1760–1849), created in late 1831 during the Edo period of Japanese history. The print depicts three boats moving through a storm-tossed sea, with a large, cresting wave forming a spiral in the centre over the boats and Mount Fuji in the background.
The print is Hokusai's best-known work and the first in his series Thirty-six Views of Mount Fuji, in which the use of Prussian blue revolutionised Japanese prints. The composition of The Great Wave is a synthesis of traditional Japanese prints and use of graphical perspective developed in Europe, and earned him immediate success in Japan and later in Europe, where Hokusai's art inspired works by the Impressionists. Several museums throughout the world hold copies of The Great Wave, many of which came from 19th-century private collections of Japanese prints. Only about

In [32]:
def get_wiki(topic: str) -> str:
    wiki_obj = wikipediaapi.Wikipedia(
        user_agent='TopicSearch (grant.mlanier@gmail.com)',
        language='en'
    )
    page = wiki_obj.page(topic)

    return (
        page.summary
        if page.exists()
        else f"No page found on Wikipedia for {topic}"
    )


In [33]:
print(get_wiki('The Great Wave off Kanagawa'))

The Great Wave off Kanagawa (Japanese: 神奈川沖浪裏, Hepburn: Kanagawa-oki Nami Ura; lit. 'Under the Wave off Kanagawa') is a woodblock print by the Japanese ukiyo-e artist Hokusai (1760–1849), created in late 1831 during the Edo period of Japanese history. The print depicts three boats moving through a storm-tossed sea, with a large, cresting wave forming a spiral in the centre over the boats and Mount Fuji in the background.
The print is Hokusai's best-known work and the first in his series Thirty-six Views of Mount Fuji, in which the use of Prussian blue revolutionised Japanese prints. The composition of The Great Wave is a synthesis of traditional Japanese prints and use of graphical perspective developed in Europe, and earned him immediate success in Japan and later in Europe, where Hokusai's art inspired works by the Impressionists. Several museums throughout the world hold copies of The Great Wave, many of which came from 19th-century private collections of Japanese prints. Only about

In [34]:
print(get_wiki('asdgf'))

No page found on Wikipedia for asdgf


Trying to create tools to use for the LLM to use. Below is a sample of the syntax with the first example of searching Wikipedia.

In [54]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_wiki",
            "description": "Fetch the summary for a topic from Wikipedia (or return a not-found message).",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic to search Wikipedia for"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    }
]


In [55]:
base_url = 'http://coruscant.local:11434/v1'

ollama = OpenAI(base_url=base_url, api_key='ollama')

In [67]:
msg = [
    {
        'role': 'user',
        'content': 'what can you tell me about The Great Wave off Kanagawa?'
    }
] 

In [ ]:
response = ollama.chat.completions.create(
    model='qwen3:8b',
    tools=tools,
    messages= msg
)

print(response.choices[0].message.content)

In [57]:
response.dict()

C:\Users\grant\AppData\Local\Temp\ipykernel_21644\3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  response.dict()


{'id': 'chatcmpl-292',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': '',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_e0m04a8z',
      'function': {'arguments': '{"topic":"The Great Wave off Kanagawa"}',
       'name': 'get_wiki'},
      'type': 'function',
      'index': 0}],
    'reasoning': 'Okay, the user is asking about "The Great Wave off Kanagawa." I remember that\'s a famous woodblock print. Let me think. It\'s by Hokusai, right? The title might be "Fujiwara no Michikatsu" or something similar. Wait, no, the correct title is "The Great Wave off Kanagawa" (Kanagawa Okino Fujō). The artwork depicts a massive wave with boats in the foreground and Mount Fuji in the background. It\'s part of the Thirty-Six Views of Mount Fuji series. Hokusai created this in the Edo period. The wave is actually a tsunami, and the composit

In [66]:
msg = [
    {
        'role': 'user',
        'content': 'what can you tell me about The Great Wave off Kanagawa?'
    }
]

In [ ]:
response = ollama.chat.completions.create(
    model='qwen3:8b',
    messages= msg    
)

print(response.choices[0].message.content)

**"The Great Wave off Kanagawa"** (Kanagawa Okinawamura / Kanagawa Okinawa-maru) is one of the most iconic and celebrated woodblock prints in the world. Here's a detailed overview:

---

### **Artist and Context**
- **Created by**: **Katsushika Hokusai** (1760–1849), a master of **ukiyo-e** (Japanese woodblock print art), known for his intricate compositions and innovative use of perspective.
- **Series**: Part of **"Thirty-Six Views of Mount Fuji"** (1831–1832), a collection of 46 prints celebrating the Japanese mountain. The wave is the fourth print in the series.
- **Time Period**: Created during Japan's **Edo period** (1603–1868), a time of relative peace and cultural flourishing.

---

### **Artistic Description**
- **Composition**: The print features a **dramatic, curling wave** (a stylized representation of a tsunami or storm) that looms over three small fishing boats. The wave's crest is rendered with **bold, swirling lines**, creating a sense of motion and power.
- **Foregroun

In [49]:
response.dict()

C:\Users\grant\AppData\Local\Temp\ipykernel_21644\3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  response.dict()


{'id': 'chatcmpl-664',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '**"The Great Wave off Kanagawa"** (Kanagawa Okinawamura / Kanagawa Okinawa-maru) is one of the most iconic and celebrated woodblock prints in the world. Here\'s a detailed overview:\n\n---\n\n### **Artist and Context**\n- **Created by**: **Katsushika Hokusai** (1760–1849), a master of **ukiyo-e** (Japanese woodblock print art), known for his intricate compositions and innovative use of perspective.\n- **Series**: Part of **"Thirty-Six Views of Mount Fuji"** (1831–1832), a collection of 46 prints celebrating the Japanese mountain. The wave is the fourth print in the series.\n- **Time Period**: Created during Japan\'s **Edo period** (1603–1868), a time of relative peace and cultural flourishing.\n\n---\n\n### **Artistic Description**\n- **Composition**: The print features a **dramatic, curling wave** (a stylized representation of a tsunami or storm) that looms over

It looks like I was able to pass the tool to the model but it did not use it. Assuming this is becuase it was able to reason enough within its training data. I would need to find a way to prompt the LLM for something that it would have to pull from Wikipedia.

In [65]:
response.choices[0].message.tool_calls[0].function.arguments

'{"topic":"The Great Wave off Kanagawa"}'

In [68]:
for tool_call in response.choices[0].message.tool_calls:
    if tool_call.function.name == 'get_wiki':
        args = json.loads(tool_call.function.arguments)
        topic = get_wiki(args['topic'])

        msg.append(
            {
                'role': 'tool',
                'tool_call_id': tool_call.id,
                'content': json.dumps({'topic': topic})
            }
        )

response = ollama.chat.completions.create(
    model='qwen3:8b',
    messages=msg,
    tools=tools
)

print(response.choices[0].message.content)

"The Great Wave off Kanagawa" (Japanese: *Kanagawa-oki Nami Ura*) is a iconic woodblock print by the 19th-century Japanese artist Katsushika Hokusai (1760–1849), created in 1831 during Japan’s Edo period. It is the first print in Hokusai’s series *Thirty-six Views of Mount Fuji* and showcases his mastery of ukiyo-e (woodblock printing).  

The composition features a towering, curling wave—symbolizing the raw power of nature—threatening three boats adrift in a stormy sea. In the background, the serene silhouette of Mount Fuji contrasts with the chaotic waves, embodying the harmony between natural elements. Hokusai’s use of **Prussian blue** (a newly introduced pigment) and European-inspired linear perspective revolutionized Japanese printmaking, blending traditional Japanese aesthetics with global artistic trends.  

This print became a cultural phenomenon, inspiring Western artists like **Vincent van Gogh**, **Claude Monet**, and **Claude Debussy**, who referenced it in their works. To

In [83]:
today = wiki_obj.page('january_24')

today_secs = today.sections
# type(today_secs[0])
today_secs[0].section_by_title('Pre-1600')


Section: Pre-1600 (2):
41 – Claudius is proclaimed Roman emperor by the Praetorian Guard after they assassinate the previous emperor, his nephew Caligula.
914 – Start of the First Fatimid invasion of Egypt.
1438 – The Council of Basel suspends Pope Eugene IV.
1458 – Matthias Corvinus is elected King of Hungary.
1536 – King Henry VIII of England suffers an accident while jousting, leading to a brain injury that historians say may have influenced his later erratic behaviour and possible impotence.
Subsections (0):

In [97]:
for sec in today_secs:
    print(sec.title)

Events
Births
Deaths
Holidays and observances
References
External links


In [103]:
for sec in today_secs[0].sections:
    print(sec.text + '\n\n')

41 – Claudius is proclaimed Roman emperor by the Praetorian Guard after they assassinate the previous emperor, his nephew Caligula.
914 – Start of the First Fatimid invasion of Egypt.
1438 – The Council of Basel suspends Pope Eugene IV.
1458 – Matthias Corvinus is elected King of Hungary.
1536 – King Henry VIII of England suffers an accident while jousting, leading to a brain injury that historians say may have influenced his later erratic behaviour and possible impotence.


1651 – Arauco War: Spanish and Mapuche authorities meet in the Parliament of Boroa renewing the fragile peace established at the parliaments of Quillín in 1641 and 1647.
1679 – King Charles II of England dissolves the Cavalier Parliament.
1742 – Charles VII Albert becomes Holy Roman Emperor.
1758 – During the Seven Years' War the leading burghers of Königsberg submit to Elizabeth of Russia, thus forming Russian Prussia (until 1763).
1817 – Crossing of the Andes: Many soldiers of Juan Gregorio de las Heras are captu

In [107]:
today_secs[0].sections[0].text.split('\n')

['41 – Claudius is proclaimed Roman emperor by the Praetorian Guard after they assassinate the previous emperor, his nephew Caligula.',
 '914 – Start of the First Fatimid invasion of Egypt.',
 '1438 – The Council of Basel suspends Pope Eugene IV.',
 '1458 – Matthias Corvinus is elected King of Hungary.',
 '1536 – King Henry VIII of England suffers an accident while jousting, leading to a brain injury that historians say may have influenced his later erratic behaviour and possible impotence.']

In [128]:
wiki_obj = wikipediaapi.Wikipedia(user_agent='TopicSearch (grant.mlanier@gmail.com)', language='en')
cur_event = wiki_obj.page('Portal:Current events')

# cur_event.exists()
print(cur_event.text)
cur_event.fullurl

'https://en.wikipedia.org/wiki/Portal:Current_events'

In [129]:
len(cur_event.summary)

0

In [130]:
[s.title for s in cur_event.sections]

[]

In [131]:
print(cur_event.exists())
print(cur_event.fullurl)
print("summary len:", len(cur_event.summary or ""))
print("text len:", len(cur_event.text or ""))
print("sections:", [s.title for s in cur_event.sections][:10])


True
https://en.wikipedia.org/wiki/Portal:Current_events
summary len: 0
text len: 0
sections: []


It appears that the 'Current Events' page does not return a value using the wikipedia api because of some issue with templates/transclusions(?) The suggestion is to use a direct call to the MediaWiki API using requests and bs4